In [1]:
from io import BytesIO
import base64
from pathlib import Path

In [2]:
import pandas as pd
import numpy as np
from transformers import ViTImageProcessor, ViTModel, ViTFeatureExtractor
from PIL import Image
import torch
import requests
import tqdm.autonotebook as tqdm

2024-04-09 21:31:08.345812: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-09 21:31:08.345872: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-09 21:31:08.347355: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
BATCH_SIZE = 64

In [4]:
df = pd.read_csv('/kaggle/input/nto-cv-olympiad-final-dataset/merged.csv')

In [5]:
def img_from_base64(data: str) -> Image.Image:
    return Image.open(BytesIO(base64.b64decode(data, validate=False)))

In [6]:
images = list(df.image.apply(img_from_base64))

In [7]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k', device_map='auto')

# inputs = processor(images=image, return_tensors="pt")
# outputs = model(**inputs)
# last_hidden_states = outputs.last_hidden_state

In [8]:
result = np.zeros((0, 768))
for i in tqdm.trange(0, len(images), BATCH_SIZE):
    img_batch = images[i:i+BATCH_SIZE]
    inputs = processor(images=img_batch, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
    features = outputs.pooler_output.cpu().numpy()
    result = np.concatenate((result, features), axis=0)

  0%|          | 0/239 [00:00<?, ?it/s]

In [10]:
!rm image_embeddings.zip
np.save('image_embeddings', result)
!zip image_embeddings.zip -xi image_embeddings.npy
!rm image_embeddings.npy

  adding: image_embeddings.npy (deflated 48%)
